In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Function to fetch historical data
def fetch_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    if len(stock_data) == 0:
        print(f"No data fetched for {symbol}. Please check the ticker symbol or date range.")
        return None
    return stock_data['Close']

# Pairs Trading Strategy Implementation
def pairs_trading_strategy(stock1, stock2, window, initial_capital):
    stock1_prices = fetch_data(stock1, "2017-01-01", "2023-12-31")
    stock2_prices = fetch_data(stock2, "2017-01-01", "2023-12-31")
    
    # Check if both stocks' data are available and have enough data points You moron. there is none for LCID and TSLA as well as some issues with Google and Meta
    if stock1_prices is None or stock2_prices is None or len(stock1_prices) < window or len(stock2_prices) < window:
        print(f"Insufficient data for pair: {stock1}/{stock2}")
        return None
    
    price_ratio = stock1_prices / stock2_prices
    moving_avg = price_ratio.rolling(window=window, center=False).mean()
    moving_std = price_ratio.rolling(window=window, center=False).std()

    # Initialize variables
    capital = initial_capital
    stock1_shares = 0
    stock2_shares = 0
    capital_over_time = [initial_capital] * window
    trade_signals = []

    # Trading strategy logic
    for i in range(window, len(price_ratio)):
        z_score = (price_ratio[i] - moving_avg[i]) / moving_std[i]
        if z_score > 1:  # Sell short the ratio
            stock1_shares -= 1
            stock2_shares += 1
            capital += stock1_prices[i] - stock2_prices[i]
            trade_signals.append(('short', stock1_prices.index[i]))
        elif z_score < -1:  # Buy long the ratio
            stock1_shares += 1
            stock2_shares -= 1
            capital -= stock1_prices[i] - stock2_prices[i]
            trade_signals.append(('long', stock1_prices.index[i]))
        elif abs(z_score) <= 0.5:  # Exit positions
            capital += stock1_shares * stock1_prices[i] - stock2_shares * stock2_prices[i]
            stock1_shares = 0
            stock2_shares = 0
        capital_over_time.append(capital + stock1_shares * stock1_prices[i] + stock2_shares * stock2_prices[i])

    return {
        "final_capital": capital,
        "capital_over_time": capital_over_time,
        "price_ratio": price_ratio.values,
        "trade_signals": trade_signals
    }

# Execute and store strategy outcomes for multiple pairs
def execute_multiple_strategies(pairs_list, window, initial_capital):
    results = {}
    for pair in pairs_list:
        stock1, stock2 = pair
        print(f"Running strategy for pair: {stock1} and {stock2}")
        result = pairs_trading_strategy(stock1, stock2, window, initial_capital)
        
        # Skip if there's insufficient data for the pair
        if result is None:
            continue
        
        # Storing results
        results[f"{stock1}/{stock2}"] = result
        
        # Plotting the capital over time for each pair
        plt.figure(figsize=(12, 6))
        plt.plot(result["capital_over_time"], label=f"Capital Over Time for {stock1}/{stock2}")
        plt.title(f"Capital Over Time for {stock1}/{stock2}")
        plt.xlabel("Days")
        plt.ylabel("Capital in $")
        plt.legend()
        plt.show()
        
    return results

# Define your pairs and parameters here
pairs_list = [
    ("MSFT", "AAPL"),  # Microsoft and Apple
    ("BAC", "JPM"),    # Bank of America and JPMorgan Chase
    ("XOM", "CVX"),    # Exxon Mobil and Chevron
    ("WMT", "TGT"),    # Walmart and Target
    ("KO", "PEP"),     # Coca-Cola and PepsiCo
    ("JNJ", "ABT"),    # Johnson & Johnson and Abbott Laboratories
    ("PG", "UL"),      # Procter & Gamble and Unilever
    ("BA", "EADSY"),   # Boeing and Airbus
    ("CAT", "DE"),     # Caterpillar and John Deere
    ("WFC", "C"),      # Wells Fargo and Citigroup
]
window = 60
initial_capital = 1000

# Run the strategy on multiple pairs
results = execute_multiple_strategies(pairs_list, window, initial_capital)

# Print the final capital for each pair
for pair, result in results.items():
    print(f"Final capital for {pair}: ${result['final_capital']:.2f}")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Function to fetch historical data
def fetch_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    if len(stock_data) == 0:
        print(f"No data fetched for {symbol}. Please check the ticker symbol or date range.")
        return None
    return stock_data['Close']

# Function to calculate MACD and Signal Line
def calculate_macd(price_ratio, n_slow=26, n_fast=12, n_signal=9):
    ema_slow = price_ratio.ewm(span=n_slow, min_periods=1).mean()
    ema_fast = price_ratio.ewm(span=n_fast, min_periods=1).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=n_signal, min_periods=1).mean()
    return macd, signal_line

# Pairs Trading Strategy Implementation using MACD
def pairs_trading_strategy(stock1, stock2, initial_capital):
    stock1_prices = fetch_data(stock1, "2017-01-01", "2023-12-31")
    stock2_prices = fetch_data(stock2, "2017-01-01", "2023-12-31")
    
    if stock1_prices is None or stock2_prices is None:
        print(f"Insufficient data for pair: {stock1}/{stock2}")
        return None
    
    price_ratio = stock1_prices / stock2_prices
    macd, signal_line = calculate_macd(price_ratio)

    # Initialize variables
    capital = initial_capital
    stock1_shares = 0
    stock2_shares = 0
    capital_over_time = [initial_capital]
    trade_signals = []

    # Trading strategy logic using MACD
    for i in range(1, len(price_ratio)):
        # Buy signal (MACD crosses above Signal line)
        if macd[i-1] < signal_line[i-1] and macd[i] > signal_line[i]:
            stock1_shares += 1
            stock2_shares -= 1
            capital -= stock1_prices[i] - stock2_prices[i]
            trade_signals.append(('long', stock1_prices.index[i]))

        # Sell signal (MACD crosses below Signal line)
        elif macd[i-1] > signal_line[i-1] and macd[i] < signal_line[i]:
            stock1_shares -= 1
            stock2_shares += 1
            capital += stock1_prices[i] - stock2_prices[i]
            trade_signals.append(('short', stock1_prices.index[i]))

        # Update capital tracking
        capital_over_time.append(capital + stock1_shares * stock1_prices[i] + stock2_shares * stock2_prices[i])

    return {
        "final_capital": capital,
        "capital_over_time": capital_over_time,
        "price_ratio": price_ratio.values,
        "trade_signals": trade_signals
    }

# Execute and store strategy outcomes for multiple pairs
def execute_multiple_strategies(pairs_list, initial_capital):
    results = {}
    for pair in pairs_list:
        stock1, stock2 = pair
        print(f"Running strategy for pair: {stock1} and {stock2}")
        result = pairs_trading_strategy(stock1, stock2, initial_capital)
        
        if result is None:
            continue

        results[f"{stock1}/{stock2}"] = result
        
        plt.figure(figsize=(12, 6))
        plt.plot(result["capital_over_time"], label=f"Capital Over Time for {stock1}/{stock2}")
        plt.title(f"Capital Over Time for {stock1}/{stock2}")
        plt.xlabel("Days")
        plt.ylabel("Capital in $")
        plt.legend()
        plt.show()
        
    return results

# Define your pairs and parameters here
pairs_list = [
    ("MSFT", "AAPL"),  # Microsoft and Apple
    ("BAC", "JPM"),    # Bank of America and JPMorgan Chase
    ("XOM", "CVX"),    # Exxon Mobil and Chevron
    ("WMT", "TGT"),    # Walmart and Target
    ("KO", "PEP"),     # Coca-Cola and PepsiCo
    ("JNJ", "ABT"),    # Johnson & Johnson and Abbott Laboratories
    ("PG", "UL"),      # Procter & Gamble and Unilever
    ("BA", "EADSY"),   # Boeing and Airbus
    ("CAT", "DE"),     # Caterpillar and John Deere
    ("WFC", "C"),      # Wells Fargo and Citigroup
]
initial_capital = 1000

# Run the strategy on multiple pairs
results = execute_multiple_strategies(pairs_list, initial_capital)

# Print the final capital for each pair
for pair, result in results.items():
    print(f"Final capital for {pair}: ${result['final_capital']:.2f}")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Function to fetch historical data
def fetch_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    if len(stock_data) == 0:
        print(f"No data fetched for {symbol}. Please check the ticker symbol or date range.")
        return None
    return stock_data['Close']

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(price_ratio, window=20, num_std=2):
    rolling_mean = price_ratio.rolling(window=window).mean()
    rolling_std = price_ratio.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, lower_band

# Pairs Trading Strategy Implementation using Bollinger Bands
def pairs_trading_strategy(stock1, stock2, initial_capital):
    stock1_prices = fetch_data(stock1, "2017-01-01", "2023-12-31")
    stock2_prices = fetch_data(stock2, "2017-01-01", "2023-12-31")
    
    if stock1_prices is None or stock2_prices is None:
        print(f"Insufficient data for pair: {stock1}/{stock2}")
        return None
    
    price_ratio = stock1_prices / stock2_prices
    upper_band, lower_band = calculate_bollinger_bands(price_ratio)

    # Initialize variables
    capital = initial_capital
    stock1_shares = 0
    stock2_shares = 0
    capital_over_time = [initial_capital]
    trade_signals = []

    # Trading strategy logic using Bollinger Bands
    for i in range(20, len(price_ratio)):
        if price_ratio[i] > upper_band[i]:  # Sell signal (price above upper band)
            stock1_shares -= 1
            stock2_shares += 1
            capital += stock1_prices[i] - stock2_prices[i]
            trade_signals.append(('short', stock1_prices.index[i]))

        elif price_ratio[i] < lower_band[i]:  # Buy signal (price below lower band)
            stock1_shares += 1
            stock2_shares -= 1
            capital -= stock1_prices[i] - stock2_prices[i]
            trade_signals.append(('long', stock1_prices.index[i]))

        # Update capital tracking
        capital_over_time.append(capital + stock1_shares * stock1_prices[i] + stock2_shares * stock2_prices[i])

    return {
        "final_capital": capital,
        "capital_over_time": capital_over_time,
        "price_ratio": price_ratio.values,
        "trade_signals": trade_signals
    }

# Execute and store strategy outcomes for multiple pairs
def execute_multiple_strategies(pairs_list, initial_capital):
    results = {}
    for pair in pairs_list:
        stock1, stock2 = pair
        print(f"Running strategy for pair: {stock1} and {stock2}")
        result = pairs_trading_strategy(stock1, stock2, initial_capital)
        
        if result is None:
            continue

        results[f"{stock1}/{stock2}"] = result
        
        plt.figure(figsize=(12, 6))
        plt.plot(result["capital_over_time"], label=f"Capital Over Time for {stock1}/{stock2}")
        plt.title(f"Capital Over Time for {stock1}/{stock2}")
        plt.xlabel("Days")
        plt.ylabel("Capital in $")
        plt.legend()
        plt.show()
        
    return results

# Define your pairs and parameters here
pairs_list = [
   ("MSFT", "AAPL"),  # Microsoft and Apple
    ("BAC", "JPM"),    # Bank of America and JPMorgan Chase
    ("XOM", "CVX"),    # Exxon Mobil and Chevron
    ("WMT", "TGT"),    # Walmart and Target
    ("KO", "PEP"),     # Coca-Cola and PepsiCo
    ("JNJ", "ABT"),    # Johnson & Johnson and Abbott Laboratories
    ("PG", "UL"),      # Procter & Gamble and Unilever
    ("BA", "EADSY"),   # Boeing and Airbus
    ("CAT", "DE"),     # Caterpillar and John Deere
    ("WFC", "C"),      # Wells Fargo and Citigroup
]
initial_capital = 1000

# Run the strategy on multiple pairs
results = execute_multiple_strategies(pairs_list, initial_capital)

# Print the final capital for each pair
for pair, result in results.items():
    print(f"Final capital for {pair}: ${result['final_capital']:.2f}")


In [ ]:
import yfinance as yf

def estimate_price_impact(symbol, investment):
    # Fetch stock data
    stock_data = yf.Ticker(symbol)
    stock_info = stock_data.info
    current_price = stock_info['regularMarketPrice']
    avg_daily_volume = stock_info['averageDailyVolume10Day']  # 10-day average daily volume
    
    # Calculate the number of shares your investment would buy
    num_shares = investment / current_price
    
    # Estimate the impact
    # This is a simplification and should be replaced with a real model in practice
    impact_ratio = (num_shares / avg_daily_volume) ** 0.5  # Square-root law
    estimated_price_change = impact_ratio * current_price  # Rough estimation of price change
    
    return estimated_price_change

# Example usage for Microsoft (MSFT)
investment = 1000000  # $1,000,000 investment
symbols = ["MSFT", "AAPL", "BAC", "JPM", "XOM", "CVX", "WMT", "TGT", "KO", "PEP", "JNJ", "ABT", "PG", "UL", "BA", "EADSY", "CAT", "DE", "WFC", "C", "TSLA", "LCID", "META", "GOOGL"]

for symbol in symbols:
    price_change = estimate_price_impact(symbol, investment)
    print(f"Estimated price impact of investing $1,000,000 in {symbol}: ${price_change:.2f}")

In [ ]:
import yfinance as yf

def estimate_price_impact(ticker, investment_amount):
    # Fetching data for the ticker
    stock = yf.Ticker(ticker)
    data = stock.history(period="1mo")  # Fetching 1 month of historical data for more accurate average volume
    
    if data.empty:
        return (None, None)  # If data is not fetched successfully

    # Using the last available price as current price
    current_price = data['Close'].iloc[-1]
    # Calculating the average daily volume
    avg_daily_volume = data['Volume'].mean()

    # Number of shares you can buy with investment_amount
    num_shares = investment_amount / current_price

    # Assuming price impact is proportional to the square root of the fraction of the average daily volume
    price_impact = (num_shares / avg_daily_volume) ** 0.5 * current_price  # Simplified impact estimation

    return current_price, price_impact

# List of company ticker symbols
companies = [
    ("MSFT", "Microsoft"),
    ("AAPL", "Apple"),
    ("BAC", "Bank of America"),
    ("JPM", "JPMorgan Chase"),
    ("XOM", "Exxon Mobil"),
    ("CVX", "Chevron"),
    ("WMT", "Walmart"),
    ("TGT", "Target"),
    ("KO", "Coca-Cola"),
    ("PEP", "PepsiCo"),
    ("JNJ", "Johnson & Johnson"),
    ("ABT", "Abbott Laboratories"),
    ("PG", "Procter & Gamble"),
    ("UL", "Unilever"),
    ("BA", "Boeing"),
    ("EADSY", "Airbus"),
    ("CAT", "Caterpillar"),
    ("DE", "John Deere"),
    ("WFC", "Wells Fargo"),
    ("C", "Citigroup"),
    ("TSLA", "Tesla"),
    ("LCID", "Lucid Group"),
    ("META", "Meta Platforms"),
    ("GOOGL", "Alphabet")
]

investment_amount = 1000000  # $1,000,000 investment

for ticker, name in companies:
    current_price, price_impact = estimate_price_impact(ticker, investment_amount)
    if current_price is not None:
        print(f"{name} ({ticker}):")
        print(f"    Current Price: ${current_price:.2f}")
        print(f"    Estimated Price Impact: ${price_impact:.2f}\n")
    else:
        print(f"Data not available for {name} ({ticker})\n")
